In [ ]:
# Cell 1
# -*- coding: utf-8 -*-
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, label_binarize
import numpy as np
import torch.backends.cudnn as cudnn
from datetime import datetime
import matplotlib.pyplot as plt  # NEW: for line charts

# Global settings / reproducibility
cudnn.benchmark = True
torch.cuda.empty_cache()
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Root path to UCRArchive_2018
ROOT = r"D:\2025暑期科研\UCRArchive_2018\UCRArchive_2018"

In [ ]:
# Cell 2
def clean_and_pad_timeseries(raw_2d, min_len=8, cap_len=None, pad_value=0.0,
                             per_sample_standardize=True, fixed_len=None):
    """
    Clean time series with tail NaNs, z-score per-sample (optional), and pad/clip.

    Priority of output length:
      1) fixed_len: if not None, output length = fixed_len (force)
      2) cap_len:   if not None, output length = min(max_real_len, cap_len)
      3) otherwise, output length = max_real_len of this input batch
    """
    import numpy as np

    N, T = raw_2d.shape
    rows, keep_idx, real_lens = [], [], []

    for i in range(N):
        row = raw_2d[i]
        valid_vals = row[~np.isnan(row)]
        L = valid_vals.shape[0]
        if L < min_len:
            continue
        if per_sample_standardize:
            mu = valid_vals.mean()
            sigma = valid_vals.std()
            valid_vals = (valid_vals - mu) / (sigma if sigma > 0 else 1.0)
        rows.append(valid_vals); keep_idx.append(i); real_lens.append(L)

    if len(rows) == 0:
        raise ValueError("All samples filtered out. Lower min_len if needed.")

    max_real_len = max(real_lens)
    if fixed_len is not None:
        target_len = int(fixed_len)
    elif cap_len is not None:
        target_len = min(max_real_len, cap_len)
    else:
        target_len = max_real_len

    out = []
    for arr in rows:
        if arr.shape[0] >= target_len:
            arr = arr[:target_len]
        else:
            arr = np.pad(arr, (0, target_len - arr.shape[0]), constant_values=pad_value)
        out.append(arr)

    X = np.stack(out, axis=0).astype("float32")
    return X, np.array(keep_idx, dtype=np.int64), np.array(real_lens, dtype=np.int64)

In [ ]:
# Cell 3
class TwoTowerTransformer(nn.Module):
    """
    Two-tower Transformer:
      - Tower 1: time series tokens [B, T, 1] -> embed -> transformer
      - Tower 2: Aout vector [B, F] as a single token -> embed -> transformer
      - Concat tokens -> final transformer -> flatten -> FC for multi-class logits
    """
    def __init__(self, input_dim1, input_dim2,
                 hidden_dim1, hidden_dim2, hidden_dim3,
                 num_heads, num_layers, num_classes,
                 seq_len1, seq_len2):
        super().__init__()
        # To keep it simple we force equal hidden dims and divisibility by nhead
        assert hidden_dim1 == hidden_dim2 == hidden_dim3, "hidden dims must be equal in this version."
        for h in (hidden_dim1, hidden_dim2, hidden_dim3):
            assert h % num_heads == 0, "hidden_dim must be divisible by num_heads"

        self.embedding1 = nn.Linear(input_dim1, hidden_dim1)
        self.embedding2 = nn.Linear(input_dim2, hidden_dim2)
        self.relu = nn.ReLU()

        self.transformer1 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim1, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.transformer2 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim2, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.final_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim3, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.dropout = nn.Dropout(0.3)
        self.seq_len1 = seq_len1
        self.seq_len2 = 1  # treat Aout as a single token
        fc_in = hidden_dim3 * (seq_len1 + self.seq_len2)
        self.fc = nn.Linear(fc_in, num_classes)

    def forward(self, x1, x2):
        # x1: [B, T, input_dim1], x2: [B, F] or [B, 1, F]
        if x1.dim() == 2:
            x1 = x1.unsqueeze(-1)
        x1 = self.relu(self.embedding1(x1))
        x1 = self.transformer1(x1)

        if x2.dim() == 3:
            assert x2.size(1) == 1, "Expect x2 with L2=1 if 3D"
            x2 = x2.squeeze(1)
        x2 = self.relu(self.embedding2(x2))
        x2 = x2.unsqueeze(1)
        x2 = self.transformer2(x2)

        x = torch.cat((x1, x2), dim=1)
        x = self.final_transformer(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class VisitDataset(Dataset):
    """Simple tensor dataset for (visit time series, aout features, one-hot labels)."""
    def __init__(self, visit_x, aout_x, y):
        self.visit_x = visit_x.astype("float32")
        self.aout_x  = aout_x.astype("float32")
        self.y       = y.astype("float32")
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.visit_x[idx], self.aout_x[idx], self.y[idx]

In [ ]:
# Cell 4
# ===== Backbones: Informer / FEDformer (Encoder-only Classifiers) =====
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class LearnedPositionalEncoding(nn.Module):
    def __init__(self, seq_len: int, d_model: int, dropout: float = 0.0):
        super().__init__()
        self.pe = nn.Parameter(torch.zeros(1, seq_len, d_model))
        nn.init.trunc_normal_(self.pe, std=0.02)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1), :])

class TokenEmbedding(nn.Module):
    def __init__(self, input_dim: int, d_model: int):
        super().__init__()
        self.proj = nn.Linear(input_dim, d_model)
    def forward(self, x):  # x: [B, L, C]
        return self.proj(x)

class InformerEncoderLayer(nn.Module):
    def __init__(self, d_model=256, n_heads=4, d_ff=512, dropout=0.1, attn_dropout=0.1, activation='gelu'):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=attn_dropout, batch_first=True)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU() if activation.lower()=='gelu' else nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)
    def forward(self, x, attn_mask=None, key_padding_mask=None):
        attn_out, _ = self.self_attn(x, x, x, attn_mask=attn_mask, key_padding_mask=key_padding_mask, need_weights=False)
        x = self.norm1(x + self.dropout1(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout2(ff_out))
        return x

class DistilConv1D(nn.Module):
    def __init__(self, d_model, dropout=0.0):
        super().__init__()
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, stride=2, padding=1)
        self.norm = nn.BatchNorm1d(d_model)
        self.act = nn.ELU()
        self.drop = nn.Dropout(dropout)
    def forward(self, x):  # [B, L, D]
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.drop(x)
        return x.transpose(1, 2)

class InformerEncoder(nn.Module):
    def __init__(self, d_model=256, n_heads=4, e_layers=2, d_ff=512, dropout=0.1, distil=True, attn_dropout=0.1, activation='gelu'):
        super().__init__()
        self.layers = nn.ModuleList([
            InformerEncoderLayer(d_model, n_heads, d_ff, dropout, attn_dropout, activation)
            for _ in range(e_layers)
        ])
        self.distil_layers = nn.ModuleList([DistilConv1D(d_model, dropout=dropout) for _ in range(e_layers-1)]) if distil and e_layers>1 else None
    def forward(self, x):  # [B, L, D]
        if self.distil_layers is None:
            for enc in self.layers: x = enc(x)
            return x
        for i,enc in enumerate(self.layers):
            x = enc(x)
            if i < len(self.layers)-1: x = self.distil_layers[i](x)
        return x

class FourierBlock(nn.Module):
    def __init__(self, k_ratio=0.25):
        super().__init__()
        self.k_ratio = k_ratio
    def forward(self, x):  # [B, L, D]
        B,L,D = x.shape
        Xf = torch.fft.rfft(x, dim=1)
        k = max(1, int((L//2 + 1) * self.k_ratio))
        Xf[:, k:, :] = 0
        return torch.fft.irfft(Xf, n=L, dim=1)

class FEDEncoderLayer(nn.Module):
    def __init__(self, d_model=256, d_ff=512, dropout=0.1, activation='gelu', k_ratio=0.25):
        super().__init__()
        self.fourier = FourierBlock(k_ratio=k_ratio)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU() if activation.lower()=='gelu' else nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)
    def forward(self, x):
        z = self.fourier(x)
        x = self.norm1(x + self.dropout1(z))
        z2 = self.ff(x)
        x = self.norm2(x + self.dropout2(z2))
        return x

class FEDformerEncoder(nn.Module):
    def __init__(self, d_model=256, e_layers=2, d_ff=512, dropout=0.1, activation='gelu', k_ratio=0.25):
        super().__init__()
        self.layers = nn.ModuleList([FEDEncoderLayer(d_model, d_ff, dropout, activation, k_ratio) for _ in range(e_layers)])
    def forward(self, x):
        for enc in self.layers: x = enc(x)
        return x

class TimePoolClassifierHead(nn.Module):
    def __init__(self, d_model: int, num_classes: int, pool: str = "mean", dropout: float = 0.1):
        super().__init__()
        assert pool in ("mean","max","cls")
        self.pool = pool
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, num_classes)
        self.cls_token = nn.Parameter(torch.zeros(1,1,d_model)) if pool=="cls" else None
        if self.cls_token is not None: nn.init.trunc_normal_(self.cls_token, std=0.02)
    def forward(self, x):  # [B, L, D]
        if self.pool=="mean":
            h = x.mean(dim=1)
        elif self.pool=="max":
            h,_ = x.max(dim=1)
        else:
            h = x[:,0,:]
        h = self.dropout(h)
        return self.fc(h)

class InformerEncoderClassifier(nn.Module):
    """Drop-in 替换 TwoTower；forward(x1,x2) 忽略 x2；x1=[B,L,1]"""
    def __init__(self, input_dim:int, num_classes:int, seq_len:int,
                 d_model=256, n_heads=4, e_layers=2, d_ff=512,
                 dropout=0.1, attn_dropout=0.1, distil=True, activation='gelu', pool="mean"):
        super().__init__()
        self.token = TokenEmbedding(input_dim, d_model)
        self.pos = LearnedPositionalEncoding(seq_len, d_model, dropout)
        self.encoder = InformerEncoder(d_model, n_heads, e_layers, d_ff, dropout, distil, attn_dropout, activation)
        self.head = TimePoolClassifierHead(d_model, num_classes, pool, dropout)
    def forward(self, x1, x2=None):
        if x1.dim()==2: x1 = x1.unsqueeze(-1)
        x = self.token(x1); x = self.pos(x); x = self.encoder(x)
        return self.head(x)

class FEDformerEncoderClassifier(nn.Module):
    """Drop-in 替换 TwoTower；forward(x1,x2) 忽略 x2；x1=[B,L,1]"""
    def __init__(self, input_dim:int, num_classes:int, seq_len:int,
                 d_model=256, e_layers=2, d_ff=512, dropout=0.1, activation='gelu', k_ratio=0.25, pool="mean"):
        super().__init__()
        self.token = TokenEmbedding(input_dim, d_model)
        self.pos = LearnedPositionalEncoding(seq_len, d_model, dropout)
        self.encoder = FEDformerEncoder(d_model, e_layers, d_ff, dropout, activation, k_ratio)
        self.head = TimePoolClassifierHead(d_model, num_classes, pool, dropout)
    def forward(self, x1, x2=None):
        if x1.dim()==2: x1 = x1.unsqueeze(-1)
        x = self.token(x1); x = self.pos(x); x = self.encoder(x)
        return self.head(x)


In [ ]:
# Cell 5
def run_one_dataset(dataset_dir, dataset_name,
                    device='cuda:0',
                    batch_size=8, num_epochs=100, lr=1e-4,
                    cap_len=None, patience=15,
                    verbose=False, plot_curves=False,
                    backbone='informer',               # ← 默认 informer
                    # === for loss TSV ===
                    return_epoch_losses=False,
                    force_full_epochs=False):
    import os, numpy as np, pandas as pd
    import matplotlib.pyplot as plt
    import torch, torch.nn as nn, torch.optim as optim
    from torch.utils.data import DataLoader
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from sklearn.preprocessing import StandardScaler, label_binarize
    from sklearn.metrics import accuracy_score, roc_auc_score

    # --- 路径 ---
    tsv_train_path = os.path.join(dataset_dir, f"{dataset_name}_TRAIN_cleaned.tsv")
    tsv_test_path  = os.path.join(dataset_dir, f"{dataset_name}_TEST_cleaned.tsv")
    aout_train_csv = os.path.join(dataset_dir, f"{dataset_name}_Aout_train_k2.csv")
    aout_test_csv  = os.path.join(dataset_dir, f"{dataset_name}_Aout_test_k2.csv")
    if not all(os.path.exists(p) for p in [tsv_train_path, tsv_test_path, aout_train_csv, aout_test_csv]):
        print(f"⚠ Skip {dataset_name}, missing files"); return None

    # --- 读取 ---
    tsv_tr = pd.read_csv(tsv_train_path, sep="\t", header=None)
    tsv_te = pd.read_csv(tsv_test_path,  sep="\t", header=None)
    csv_tr = pd.read_csv(aout_train_csv, header=None)
    csv_te = pd.read_csv(aout_test_csv,  header=None)

    y_tr_raw = tsv_tr.iloc[:,0].values
    y_te_raw = tsv_te.iloc[:,0].values
    visit_tr_raw = tsv_tr.iloc[:,1:].values.astype("float32")
    visit_te_raw = tsv_te.iloc[:,1:].values.astype("float32")
    aout_tr_raw_all  = csv_tr.iloc[:,1:].values.astype("float32")
    aout_te_raw_all  = csv_te.iloc[:,1:].values.astype("float32")

    # --- 用 Train 决定参考长度 ---
    tmp_train_clean, keep_tr0, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=None
    )
    train_seq_len = tmp_train_clean.shape[1]

    # --- 固定同长清洗 ---
    visit_tr_clean, keep_tr, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_tr = y_tr_raw[keep_tr]
    aout_tr_raw = aout_tr_raw_all[keep_tr]

    visit_te_clean, keep_te, _ = clean_and_pad_timeseries(
        visit_te_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_te = y_te_raw[keep_te]
    aout_te_raw = aout_te_raw_all[keep_te]

    # --- 右塔标准化（Informer/FED 不用右塔，但保持一致做 fit/transform，DataLoader 结构不改） ---
    scaler = StandardScaler().fit(aout_tr_raw)
    aout_tr = scaler.transform(aout_tr_raw).astype("float32")
    aout_te = scaler.transform(aout_te_raw).astype("float32")

    # --- One-hot 标签 ---
    classes = sorted(np.unique(y_tr))
    Y_tr = label_binarize(y_tr, classes=classes).astype("float32")
    Y_te = label_binarize(y_te, classes=classes).astype("float32")

    # --- 左塔输入形状 [B, L, 1] ---
    visit_tr = visit_tr_clean[:, :, None]
    visit_te = visit_te_clean[:, :, None]

    # --- 维度参数 ---
    seq_len1 = visit_tr.shape[1]
    input_dim1 = visit_tr.shape[2]
    input_dim2 = aout_tr.shape[1]   # TwoTower 用；Informer/FED 忽略
    num_classes = Y_tr.shape[1]

    device_torch = torch.device(device if torch.cuda.is_available() else 'cpu')

    # ========== 构建模型 ==========
    if backbone.lower() == 'twotower':
        hidden_dim1 = hidden_dim2 = hidden_dim3 = 16
        num_heads = 2; num_layers = 2; seq_len2 = 1
        model = TwoTowerTransformer(
            input_dim1, input_dim2,
            hidden_dim1, hidden_dim2, hidden_dim3,
            num_heads, num_layers,
            num_classes,
            seq_len1, seq_len2
        ).to(device_torch)
    elif backbone.lower() == 'informer':
        model = InformerEncoderClassifier(
            input_dim=1, num_classes=num_classes, seq_len=seq_len1,
            d_model=256, n_heads=4, e_layers=2, d_ff=512,
            dropout=0.1, attn_dropout=0.1, distil=True, activation='gelu', pool='mean'
        ).to(device_torch)
    elif backbone.lower() in ('fed', 'fedformer'):
        model = FEDformerEncoderClassifier(
            input_dim=1, num_classes=num_classes, seq_len=seq_len1,
            d_model=256, e_layers=2, d_ff=512,
            dropout=0.1, activation='gelu', k_ratio=0.25, pool='mean'
        ).to(device_torch)
    else:
        raise ValueError(f"Unknown backbone: {backbone}")

    if torch.cuda.device_count() > 1 and str(device_torch).startswith('cuda'):
        model = nn.DataParallel(model, device_ids=[0,1])
    model = model.to(device_torch)

    # --- DataLoader ---
    train_loader = DataLoader(VisitDataset(visit_tr, aout_tr, Y_tr),
                              batch_size=batch_size, shuffle=True,  pin_memory=True, num_workers=0)
    test_loader  = DataLoader(VisitDataset(visit_te, aout_te, Y_te),
                              batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=0)

    # --- 优化器/损失/调度器 ---
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.0)

    # --- EarlyStopping（基于 TrainLoss） ---
    class EarlyStopping:
        def __init__(self, patience=15, delta=0.0):
            self.patience = patience; self.delta = delta
            self.counter = 0; self.best = None; self.stop = False
        def step(self, train_loss):
            if self.best is None: self.best = train_loss; return False
            if train_loss > self.best - self.delta:
                self.counter += 1
                if self.counter >= self.patience: self.stop = True
            else:
                self.best = train_loss; self.counter = 0
            return self.stop
    es = EarlyStopping(patience=patience)

    # --- 训练 + 逐 epoch 记录 TEST LOSS ---
    log_dir = os.path.join(dataset_dir, f"_{backbone.lower()}_logs")
    os.makedirs(log_dir, exist_ok=True)
    log_file = open(os.path.join(log_dir, f"log_{backbone.lower()}.txt"), "w", encoding="utf-8")

    train_loss_hist = []
    test_loss_hist  = []

    for epoch in range(num_epochs):
        # train
        model.train()
        total_loss = 0.0
        for x1,x2,y in train_loader:
            x1 = x1.to(torch.float32).to(device_torch)
            x2 = x2.to(torch.float32).to(device_torch)
            y  = y.to(torch.float32).to(device_torch)
            optimizer.zero_grad()
            logits = model(x1,x2)       # 单塔忽略 x2
            loss = criterion(logits, y)
            loss.backward(); optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        avg_train_loss = total_loss / max(1,len(train_loader))
        train_loss_hist.append(avg_train_loss)

        # test loss (每个 epoch 评估一次；只保存在 test_loss_hist 中)
        model.eval()
        total_tloss = 0.0
        with torch.no_grad():
            for x1,x2,y in test_loader:
                x1 = x1.to(torch.float32).to(device_torch)
                x2 = x2.to(torch.float32).to(device_torch)
                y  = y.to(torch.float32).to(device_torch)
                logits = model(x1,x2)
                tloss = criterion(logits, y)
                total_tloss += tloss.item()
        avg_test_loss = total_tloss / max(1,len(test_loader))
        test_loss_hist.append(avg_test_loss)

        log_file.write(f"Epoch [{epoch+1}/{num_epochs}] "
                       f"Train Loss: {avg_train_loss:.4f} | "
                       f"Test Loss: {avg_test_loss:.4f}\n")

        # 早停仅在不强制满 epoch 时才中断（早停依据仍是 train loss，与之前保持一致）
        if es.step(avg_train_loss) and not force_full_epochs:
            break

    log_file.close()

    # --- 最终 Test（一次性 AUC/ACC） ---
    model.eval()
    tout=[]; tlab=[]
    with torch.no_grad():
        for x1,x2,y in test_loader:
            x1 = x1.to(torch.float32).to(device_torch)
            x2 = x2.to(torch.float32).to(device_torch)
            tout.append(model(x1,x2).detach().cpu().numpy())
            tlab.append(y.numpy())
    tout = np.concatenate(tout); tlab = np.concatenate(tlab)
    try:
        test_auc = roc_auc_score(tlab, tout, multi_class='ovr')
    except:
        test_auc = roc_auc_score(tlab, tout)
    test_pred = np.argmax(tout, axis=1)
    test_true = np.argmax(tlab, axis=1)
    test_acc = accuracy_score(test_true, test_pred)
    test_n = len(tlab)

    print(f"[{dataset_name} | {backbone}] TEST AUC={test_auc:.4f}, TEST ACC={test_acc:.4f}, n_samples={test_n}")

    if return_epoch_losses:
        # 返回 test_loss_hist（用于 TSV）
        return test_auc, test_acc, test_n, test_loss_hist
    else:
        return test_auc, test_acc, test_n


In [ ]:
# Cell 6
def discover_datasets(root):
    """
    Discover dataset subfolders that contain all four required files:
      *_TRAIN_cleaned.tsv, *_TEST_cleaned.tsv, *_Aout_train_k2.csv, *_Aout_test_k2.csv
    Returns: a sorted list of dataset names (folder names).
    """
    names = []
    for name in sorted(os.listdir(root)):
        subdir = os.path.join(root, name)
        if not os.path.isdir(subdir):
            continue
        t_train = os.path.join(subdir, f"{name}_TRAIN_cleaned.tsv")
        t_test  = os.path.join(subdir, f"{name}_TEST_cleaned.tsv")
        a_train = os.path.join(subdir, f"{name}_Aout_train_k2.csv")
        a_test  = os.path.join(subdir, f"{name}_Aout_test_k2.csv")
        if all(os.path.exists(p) for p in [t_train, t_test, a_train, a_test]):
            names.append(name)
    return names

In [ ]:
# Cell 7
def run_all_datasets(root, device='cuda:0',
                     batch_size=8, num_epochs=100, lr=1e-4,
                     cap_len=None, verbose=False, plot_curves=True,
                     patience=15, backbone='twotower'):
    import os, pandas as pd
    from datetime import datetime

    dataset_names = discover_datasets(root)
    print(f"Found {len(dataset_names)} datasets:", dataset_names)

    rows = []
    for name in dataset_names:
        out = run_one_dataset(
            dataset_dir=os.path.join(root, name),
            dataset_name=name,
            device=device,
            batch_size=batch_size, num_epochs=num_epochs, lr=lr,
            cap_len=cap_len, patience=patience,
            verbose=verbose, plot_curves=plot_curves,
            backbone=backbone   # <-- 透传骨干
        )
        if out is None: continue
        test_auc, test_acc, test_n = out
        rows.append((name, test_auc, test_acc, test_n))

    if not rows:
        print("No dataset finished successfully."); return None

    df = pd.DataFrame(rows, columns=["dataset", "test_auc", "test_acc", "n_samples"])
    mean_auc = df["test_auc"].mean(); mean_acc = df["test_acc"].mean()
    w_auc = (df["test_auc"] * df["n_samples"]).sum() / df["n_samples"].sum()
    w_acc = (df["test_acc"] * df["n_samples"]).sum() / df["n_samples"].sum()

    print("\n========== Summary (OFFICIAL TEST) ==========")
    print(df.sort_values("dataset").to_string(index=False))
    print(f"\nSimple mean: AUC = {mean_auc:.4f}, ACC = {mean_acc:.4f}")
    print(f"Weighted (by samples): AUC = {w_auc:.4f}, ACC = {w_acc:.4f}")

    summary_dir = os.path.join(root, "_twotower_logs"); os.makedirs(summary_dir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    tag = backbone.lower()
    df.to_csv(os.path.join(summary_dir, f"summary_TEST_{tag}_{ts}.csv"), index=False, encoding="utf-8")
    with open(os.path.join(summary_dir, f"summary_TEST_{tag}_{ts}.txt"), "w", encoding="utf-8") as f:
        f.write(df.sort_values("dataset").to_string(index=False))
        f.write(f"\n\nSimple mean: AUC={mean_auc:.6f}, ACC={mean_acc:.6f}\n")
        f.write(f"Weighted (by samples): AUC={w_auc:.6f}, ACC={w_acc:.6f}\n")

    return df, (mean_auc, mean_acc), (w_auc, w_acc)

In [ ]:
# Cell 8
# === BATCH COLLECTOR (Informer): build logs/informer_testloss.tsv ===
import os, numpy as np, pandas as pd

NUM_EPOCHS = 60
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)

dataset_names = discover_datasets(ROOT)
print(f"[Informer] Discover {len(dataset_names)} datasets")

loss_table = pd.DataFrame(index=np.arange(1, NUM_EPOCHS+1),
                          columns=dataset_names, dtype=float)
loss_table.index.name = "epoch"

for ds in dataset_names:
    try:
        out = run_one_dataset(
            dataset_dir=os.path.join(ROOT, ds),
            dataset_name=ds,
            device='cuda:0',
            batch_size=8,
            num_epochs=NUM_EPOCHS,
            lr=1e-4,
            cap_len=None,
            patience=15,
            verbose=False,
            plot_curves=False,
            return_epoch_losses=True,   # ← 取回逐 epoch 的 test loss
            force_full_epochs=True,     # ← 强制跑满 60 个 epoch
            backbone='informer'
        )
        if out is None:
            print(f"[SKIP] {ds} returned None")
            loss_table[ds] = np.nan
            continue

        _, _, _, epoch_losses = out
        if len(epoch_losses) != NUM_EPOCHS:
            print(f"[WARN] {ds}: got {len(epoch_losses)} epochs, expect {NUM_EPOCHS}. Pad/Crop.")
        epoch_losses = (list(epoch_losses) + [np.nan]*NUM_EPOCHS)[:NUM_EPOCHS]
        loss_table[ds] = epoch_losses

    except Exception as e:
        print(f"[ERROR] {ds}: {e}")
        loss_table[ds] = np.nan

out_path = os.path.join(LOG_DIR, "informer_testloss.tsv")
loss_table.to_csv(out_path, sep="\t", float_format="%.6f")
print(f"[Informer] Saved test-loss table to {out_path} with shape {loss_table.shape}")




[Informer] Discover 125 datasets
[ACSF1 | informer] TEST AUC=0.9680, TEST ACC=0.7700, n_samples=100
[Adiac | informer] TEST AUC=0.9518, TEST ACC=0.3606, n_samples=391
[AllGestureWiimoteX | informer] TEST AUC=0.8007, TEST ACC=0.3084, n_samples=681
[AllGestureWiimoteY | informer] TEST AUC=0.8175, TEST ACC=0.3736, n_samples=645
[AllGestureWiimoteZ | informer] TEST AUC=0.7978, TEST ACC=0.3372, n_samples=685
[ArrowHead | informer] TEST AUC=0.7472, TEST ACC=0.5086, n_samples=175
[BME | informer] TEST AUC=0.8010, TEST ACC=0.5867, n_samples=150
[Beef | informer] TEST AUC=0.7569, TEST ACC=0.4667, n_samples=30
[BeetleFly | informer] TEST AUC=0.8700, TEST ACC=1.0000, n_samples=20
[BirdChicken | informer] TEST AUC=0.8800, TEST ACC=1.0000, n_samples=20
[CBF | informer] TEST AUC=0.8399, TEST ACC=0.6278, n_samples=900
[Car | informer] TEST AUC=0.7136, TEST ACC=0.5333, n_samples=60
[Chinatown | informer] TEST AUC=0.9837, TEST ACC=1.0000, n_samples=343
[ChlorineConcentration | informer] TEST AUC=0.6094